In [2]:
%load_ext autoreload
%autoreload 2
from davidyu_cfg import *
from functions.baostock.stock_return import *
from functions.common.loadModule.load_module_kdj import *
import matplotlib.pyplot as plt

df1 = pd.read_csv("/home/davidyu/stock/data/history_data/dfcf_fuquan/stock_index/600787_fuquan.csv")
df2 = df1[df1["dt"]>="2018-01-01"]
# df2["volume"].rolling(3).mean().plot(figsize=(20, 6), style='b')

# df2["volume"].plot( style='b*-')
# df2["close"].rolling(5).mean().plot(secondary_y=True, style='g')
# df2["close"].rolling(10).mean().plot(secondary_y=True, style='y')
# df2["close"].plot(secondary_y=True, style='r*-')
df2.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,dt,open,close,high,low,volume,money,x1,x2,x3,turnover_rate,stock_date,stock_index
4973,2018-01-02,10.92,11.22,11.25,10.89,180310,202768681.0,3.30,2.75,0.30,0.97,2018-01-02,600787
4974,2018-01-03,11.23,11.21,11.28,11.12,152639,172247120.0,1.43,-0.09,-0.01,0.82,2018-01-03,600787
4975,2018-01-04,11.18,11.17,11.29,11.02,157406,176686356.0,2.41,-0.36,-0.04,0.85,2018-01-04,600787
4976,2018-01-05,11.19,11.36,11.52,11.14,304560,349503792.0,3.40,1.70,0.19,1.64,2018-01-05,600787
4977,2018-01-08,11.32,11.24,11.37,11.08,185355,208933616.0,2.55,-1.06,-0.12,1.00,2018-01-08,600787


In [61]:
df = df2
kdj_day = 9
k_stat = 3
d_stat = 3
j_stat = 2
def new_kdj(df,kdj_day=9,k_stat=3,d_stat=3):
    low_list = df['low'].rolling(kdj_day, min_periods=kdj_day).min()
    low_list.fillna(value = df['low'].expanding().min(), inplace = True)
    high_list = df['high'].rolling(kdj_day, min_periods=kdj_day).max()
    high_list.fillna(value = df['high'].expanding().max(), inplace = True)

    rsv = (df['close'] - low_list) / (high_list - low_list) * 100
    df['K'] = pd.DataFrame(rsv).ewm(com=k_stat-1).mean()
    df['D'] = df['K'].ewm(com=d_stat-1).mean()
    df['J'] = 3 * df['K'] - 2 * df['D']
    return df


df = new_kdj(df,18,6,6)  # 卖点
df["k_18_6_6"]= df["K"]
df["d_18_6_6"]= df["D"]

df = new_kdj(df,3,3,6)
df["k_3_3_3"]= df["K"]
df["d_3_3_3"]= df["D"]

df = new_kdj(df,45,15,15)
df["k_45_15_15"]= df["K"]
df["d_45_15_15"]= df["D"]

df = new_kdj(df,5,3,3)
df["k_5_3_3"]= df["K"]
df["d_5_3_3"]= df["D"]

In [1]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
# df2 = df1[df1["dt"]>="2020-03-01"]
from plotly.subplots import make_subplots
df2 = df
layout = dict(title_x=0.5, # title居中
          font=dict(family="Times New Roman",size=20,color="RebeccaPurple"),
          # 字体设置
          coloraxis_colorbar=dict(xanchor="left",x=0.75,ticks="outside"),
          # 颜色条设置
          margin=dict(b= 0,l=20, r=0, t= 0),
          width=1000,
          height=800
          # 大小设置
         )
fig_all = make_subplots(rows=4,  # 将画布分为两行
        cols=1,  # 将画布分为两列
        subplot_titles=["", 
                        ""
                     ],  # 子图的标题
        x_title="",
        y_title="y轴标题", shared_xaxes=True
       )


# trace0 = go.Candlestick(
#     name="close",
#     x=df2["dt"], low=df2["low"],high=df2["high"],
#     open=df2["open"],close=df2["close"],increasing_line_color= "red", 
#                       decreasing_line_color='green'
# )

trace0 = go.Scatter(
    name="close",
    x=df2["dt"], y=df2["close"],mode='markers+lines'
)
trace1 = go.Scatter(
    name="k_18_6_6",
    x=df2["dt"], y=df2["k_18_6_6"]
)

trace2 = go.Scatter(
     name="d_18_6_6",
     x=df2["dt"], y=df2["d_18_6_6"],
)
trace3 = go.Scatter(
     name="k_45_15_15",
     x=df2["dt"], y=df2["k_45_15_15"],
)

trace4 = go.Scatter(
     name="d_45_15_15",
     x=df2["dt"], y=df2["d_45_15_15"],
)
trace5 = go.Scatter(
     name="k_5_3_3",
     x=df2["dt"], y=df2["k_5_3_3"],
)

trace6 = go.Scatter(
     name="d_5_3_3",
     x=df2["dt"], y=df2["d_5_3_3"],
)


# fig = px.bar(df2, x='dt', y="macdh_volume")
a1=tuple([trace1,trace2])
a2=tuple([trace3,trace4])
a3=tuple([trace5,trace6])
fig_all.append_trace(trace0,1,1)  # 将trace0添加到第一行第一列的位置
#fig_all.append_trace(trace1,2,1)  # 将trace0添加到第一行第一列的位置
fig_all.add_traces(a1,2,1)
fig_all.add_traces(a2,3,1)
fig_all.add_traces(a3,4,1)

#fig_all.append_trace(trace2,3,1)  # 将trace0添加到第一行第一列的位置
# fig_all.append_trace(trace3,4,1)  # 将trace0添加到第一行第一列的位置

fig_all['layout']['yaxis1'].update(domain=[0.8, 1])
fig_all['layout']['yaxis2'].update(domain=[0.6, 0.75])
fig_all['layout']['yaxis3'].update(domain=[0.4, 0.55])
fig_all['layout']['yaxis4'].update(domain=[0.1, 0.35])



fig_all.update_layout(layout)
fig_all.update_layout(xaxis1_rangeslider_visible=False)
fig_all.update_layout(xaxis2_rangeslider_visible=False)
fig_all.update_layout(xaxis4_rangeslider_visible=True)

fig_all.show()
# fig.show()

NameError: name 'df' is not defined

In [55]:
tuple([trace0,trace1])

(Scatter({
     'name': 'close',
     'x': array(['2018-01-02', '2018-01-03', '2018-01-04', ..., '2020-12-25',
                 '2020-12-28', '2020-12-29'], dtype=object),
     'y': array([11.22, 11.21, 11.17, ...,  4.69,  4.57,  4.62])
 }), Scatter({
     'name': 'macdh_close_raw',
     'x': array(['2018-01-02', '2018-01-03', '2018-01-04', ..., '2020-12-25',
                 '2020-12-28', '2020-12-29'], dtype=object),
     'y': array([91.66666667, 86.42191142, 79.92533108, ..., 46.7537425 , 42.77097589,
                 41.8329561 ])
 }))

In [8]:
def kdj_x(x):
    stock = DF_to_StockDataFrame(x)
    ss,tt = stock_kdj(stock)
    #print(ss)
    return ss
df1 = kdj_x(df)
df1

2021-01-27 21:43:21,756 - stockstats.py[line:831] - CRITICAL: NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


,date,dt,open,close,high,low,volume,money,x1,x2,...,rsi_24,kdjk,kdjd,kdjj,boll_ub,boll_lb,macd,macds,wr_6,wr_10
0,2018-01-02,2018-01-02,10.92,11.22,11.25,10.89,180310,202768681.0,3.30,2.75,...,NaN,63.888889,54.629630,82.407407,NaN,NaN,0.000000,0.000000,8.333333,8.333333
1,2018-01-03,2018-01-03,11.23,11.21,11.28,11.12,152639,172247120.0,1.43,-0.09,...,0.000000,69.943020,59.734093,90.360874,11.229142,11.200858,-0.000224,-0.000125,17.948718,17.948718
2,2018-01-04,2018-01-04,11.18,11.17,11.29,11.02,157406,176686356.0,2.41,-0.36,...,0.000000,69.962013,63.143400,83.599240,11.252915,11.147085,-0.001526,-0.000699,30.000000,30.000000
3,2018-01-05,2018-01-05,11.19,11.36,11.52,11.14,304560,349503792.0,3.40,1.70,...,79.994152,71.509067,65.931956,82.663290,11.405731,11.074269,0.004779,0.001157,25.396825,25.396825
4,2018-01-08,2018-01-08,11.32,11.24,11.37,11.08,185355,208933616.0,2.55,-1.06,...,52.379872,66.191230,66.018380,66.536929,11.383527,11.096473,0.003329,0.001803,44.444444,44.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,2020-12-23,2020-12-23,4.73,4.74,4.77,4.71,73830,34968678.0,1.27,0.42,...,50.965171,63.065962,51.884635,85.428615,4.881116,4.529884,-0.008639,-0.016022,28.571429,28.571429
724,2020-12-24,2020-12-24,4.72,4.70,4.75,4.66,58807,27657615.0,1.90,-0.84,...,49.137110,62.043975,55.271082,75.589760,4.874974,4.529026,-0.007274,-0.014273,50.000000,40.000000
725,2020-12-25,2020-12-25,4.64,4.69,4.77,4.63,90569,42556698.0,2.98,-0.21,...,48.681592,60.410269,56.984144,67.262518,4.864972,4.529028,-0.006919,-0.012802,53.571429,42.857143
726,2020-12-28,2020-12-28,4.72,4.57,4.72,4.55,108344,50111179.0,3.62,-2.56,...,43.618339,47.892560,53.953616,35.770448,4.851135,4.517865,-0.016135,-0.013469,93.103448,77.142857


In [14]:
def kdj_x(x):
    stock = DF_to_StockDataFrame(x)
    ss,tt = stock_kdj(stock)
    #print(ss)
    return ss
def macd_col(df2,col,MACD_EMA_SHORT = 12,MACD_EMA_LONG = 26, MACD_EMA_SIGNAL = 9):
    fast=df2[col].ewm(
                ignore_na=False, span=MACD_EMA_SHORT,
                min_periods=0, adjust=True).mean()
    slow=df2[col].ewm(
                ignore_na=False, span=MACD_EMA_LONG,
                min_periods=0, adjust=True).mean()
    # df2["volume_26"].plot(secondary_y=True, style='r*-')
    # df2["volume_12"].plot(secondary_y=True, style='b*-')
    df2['macd'] = fast-slow
    df2['macds'] = df2["macd"].ewm(
                ignore_na=False, span=MACD_EMA_SIGNAL,
                min_periods=0, adjust=True).mean()
    df2['macdh'] = (df2['macd'] - df2['macds'])
    return df2

In [37]:
# fast=df2["close"].ewm(
#                 ignore_na=False, span=12,
#                 min_periods=0, adjust=True).mean()
# slow=df2["close"].ewm(
#                 ignore_na=False, span=26,
#                 min_periods=0, adjust=True).mean()
# df2['macd'] = fast-slow
# df2['macds'] = df2["macd"].ewm(
#             ignore_na=False, span=9,
#             min_periods=0, adjust=True).mean()
# df2['macdh'] = (df2['macd'] - df2['macds'])/df2['macd']
# df2['macdh'].max()

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

29.585507558106485

In [24]:

df2 = macd_col(df2,"volume",5,34,5)
df2["macdh_volume"] = df2["macdh"]
df2 = macd_col(df2,"close",34,89,34)
df2["macdh_close"] = df2["macdh"]
df2 = macd_col(df2,"close",12,26,9)
df2["macdh_close_raw"] = df2["macdh"]


/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [25]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
# df2 = df1[df1["dt"]>="2020-03-01"]
from plotly.subplots import make_subplots

layout = dict(title_x=0.5, # title居中
          font=dict(family="Times New Roman",size=20,color="RebeccaPurple"),
          # 字体设置
          coloraxis_colorbar=dict(xanchor="left",x=0.75,ticks="outside"),
          # 颜色条设置
          margin=dict(b= 0,l=20, r=0, t= 0),
          width=1000,
          height=800
          # 大小设置
         )
fig_all = make_subplots(rows=4,  # 将画布分为两行
        cols=1,  # 将画布分为两列
        subplot_titles=["", 
                        ""
                     ],  # 子图的标题
        x_title="",
        y_title="y轴标题", shared_xaxes=True
       )


# trace0 = go.Candlestick(
#     name="close",
#     x=df2["dt"], low=df2["low"],high=df2["high"],
#     open=df2["open"],close=df2["close"],increasing_line_color= "red", 
#                       decreasing_line_color='green'
# )

trace0 = go.Scatter(
    name="close",
    x=df2["dt"], y=df2["close"]
)
trace1 = go.Bar(
    name="macdh_close_raw",
    x=df2["dt"], y=df2["macdh_close_raw"]
)



trace2 = go.Bar(
     name="macdh_close",
     x=df2["dt"], y=df2["macdh_close"],
)
trace3 = go.Bar(
     name="macdh_volume",
     x=df2["dt"], y=df2["macdh_volume"],
)

# fig = px.bar(df2, x='dt', y="macdh_volume")

fig_all.append_trace(trace0,1,1)  # 将trace0添加到第一行第一列的位置
fig_all.append_trace(trace1,2,1)  # 将trace0添加到第一行第一列的位置
fig_all.append_trace(trace2,3,1)  # 将trace0添加到第一行第一列的位置
fig_all.append_trace(trace3,4,1)  # 将trace0添加到第一行第一列的位置

fig_all['layout']['yaxis1'].update(domain=[0.8, 1])
fig_all['layout']['yaxis2'].update(domain=[0.6, 0.75])
fig_all['layout']['yaxis3'].update(domain=[0.4, 0.55])
fig_all['layout']['yaxis4'].update(domain=[0.1, 0.35])



fig_all.update_layout(layout)
fig_all.update_layout(xaxis1_rangeslider_visible=False)
fig_all.update_layout(xaxis2_rangeslider_visible=False)
fig_all.update_layout(xaxis4_rangeslider_visible=True)

fig_all.show()
# fig.show()

In [63]:


# df2 = macd_col(df2,"close")
# # df2["close"].plot(figsize=(20, 6),style='b*-')
# fig, axes = plt.subplots(2, 1) 
# df2["close"].plot(kind='line', ax=axes[0],figsize=(20, 6),style='r*-')
# # fig, axes = plt.subplots(2, 2) 
# df2["macdh"].plot(kind='bar', ax=axes[1],figsize=(20, 6), style='r*-')

In [126]:


# # df2["close"].plot(figsize=(20, 6),style='b*-')
# fig, axes = plt.subplots(3, 1) 
# df2["low"].plot(kind='line', ax=axes[0],figsize=(20, 6),style='r*-')
# # fig, axes = plt.subplots(2, 2) 
# df2["macdh_volume"].plot(kind='bar', ax=axes[1],figsize=(20, 6), style='r*-')
# df2["macdh_close"].plot(kind='bar', ax=axes[2],figsize=(20, 6), style='r*-')

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [101]:
import chart_studio.plotly as py
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

df2 = df1[df1["dt"]>="2020-03-01"]
fig = px.line(df2, x='dt', y='close', title='Time Series with Rangeslider')
# fig = px.line(df2, x='dt', y='volume', title='volume')

fig.add_trace(go.Scatter(
    name="End-aligned",
    mode="markers+lines", x="dt", y="low",
    xperiod="M1",
    xperiodalignment="end"
))
# fig = px.bar(df2, x='dt', y="macdh_volume")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

AttributeError: 'Figure' object has no attribute 'line'

In [255]:
go.Candlestick


# fig = px.line(df2, x='dt', y='volume', title='volume')
# fig = go.Figure(2,1)

# fig.add_trace(go.Scatter(
#     name="Raw Data",
#     mode="markers+lines", x=df2["dt"], y=df2["close"],
#     marker_symbol="star"
# ))


<bound method BaseFigure.update_layout of Figure({
    'data': [{'marker': {'symbol': 'star'},
              'mode': 'markers+lines',
              'name': 'close',
              'type': 'scatter',
              'x': array(['2020-03-02', '2020-03-03', '2020-03-04', ..., '2020-12-25',
                          '2020-12-28', '2020-12-29'], dtype=object),
              'xaxis': 'x',
              'y': array([6.57, 6.77, 6.91, ..., 4.85, 4.78, 4.83]),
              'yaxis': 'y'},
             {'name': 'volume',
              'type': 'bar',
              'x': array(['2020-03-02', '2020-03-03', '2020-03-04', ..., '2020-12-25',
                          '2020-12-28', '2020-12-29'], dtype=object),
              'xaxis': 'x2',
              'y': array([304434, 496533, 329602, ...,  50486,  45390,  45989]),
              'yaxis': 'y2'},
             {'name': 'macdh_volume',
              'type': 'bar',
              'x': array(['2020-03-02', '2020-03-03', '2020-03-04', ..., '2020-12-25',
       

In [ ]:
fig.add_trace(go.Scatter(
    name="test",
    mode="markers+lines", x=df2["dt"], y=df2["open"],marker_symbol="star"))

fig.add_trace(go.Bar(
    name="Middle-aligned",
    x=df2["dt"], y=df2["macdh_volume"],
    xperiod="M1",
    xperiodalignment="middle"
))

In [122]:
df2["macdh_volume"]

4677       0.000000
4678    1915.517094
4679   -1227.087430
4680    -664.291973
4681   -3692.807408
           ...     
4877   -2098.629772
4878    4966.850624
4879    3192.295540
4880    1651.368577
4881     697.330873
Name: macdh_volume, Length: 205, dtype: float64

In [ ]:

MACD_EMA_SHORT = 12
MACD_EMA_LONG = 26
MACD_EMA_SIGNAL = 9

def _get_macd(cls, df):
    """ Moving Average Convergence Divergence
    This function will initialize all following columns.
    MACD Line (macd): (12-day EMA - 26-day EMA)
    Signal Line (macds): 9-day EMA of MACD Line
    MACD Histogram (macdh): MACD Line - Signal Line
    :param df: data
    :return: None
    """
    ema_short = 'close_{}_ema'.format(cls.MACD_EMA_SHORT)
    ema_long = 'close_{}_ema'.format(cls.MACD_EMA_LONG)
    ema_signal = 'macd_{}_ema'.format(cls.MACD_EMA_SIGNAL)
    fast = df[ema_short]
    slow = df[ema_long]
    df['macd'] = fast - slow
    df['macds'] = df[ema_signal]
    df['macdh'] = (df['macd'] - df['macds'])
    cls._drop_columns(df, [ema_short, ema_long, ema_signal])

c='close'
r='9'
t="ema"


@classmethod
def _get_ema(cls, df, column, windows):
    """ get exponential moving average
    :param df: data
    :param column: column to calculate
    :param windows: collection of window of exponential moving average
    :return: None
    """
    window = cls.get_only_one_positive_int(windows)
    column_name = '{}_{}_ema'.format(column, window)
    if len(df[column]) > 0:
        df[column_name] = df[column].ewm(
            ignore_na=False, span=window,
            min_periods=0, adjust=True).mean()
    else:
        df[column_name] = []